In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.2/transformers/1b-instruct/1/config.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/README.md
/kaggle/input/llama-3.2/transformers/1b-instruct/1/USE_POLICY.md
/kaggle/input/llama-3.2/transformers/1b-instruct/1/tokenizer.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/tokenizer_config.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/LICENSE.txt
/kaggle/input/llama-3.2/transformers/1b-instruct/1/model.safetensors
/kaggle/input/llama-3.2/transformers/1b-instruct/1/special_tokens_map.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/.gitattributes
/kaggle/input/llama-3.2/transformers/1b-instruct/1/generation_config.json


In [2]:
! pip install langchain

In [4]:
! pwd

/kaggle/working


In [5]:
! ls

In [6]:
! ls - /kaggle/input/llama-3.2/transformers/1b-instruct/1

ls: cannot access '-': No such file or directory
/kaggle/input/llama-3.2/transformers/1b-instruct/1:
config.json		model.safetensors	 tokenizer_config.json
generation_config.json	README.md		 tokenizer.json
LICENSE.txt		special_tokens_map.json  USE_POLICY.md


In [7]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [10]:
#model directory
model_directory = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"

# Load the tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(model_directory)
    print("Tokenizer loaded successfully!")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print(f"Please ensure the tokenizer files are in: {model_directory}")

# Load the model
try:
    model = AutoModelForCausalLM.from_pretrained(model_directory).to(device)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    print(f"Please ensure the model files are in: {model_directory}")



Tokenizer loaded successfully!
Model loaded successfully!


In [21]:
if tokenizer and model:
    prompt = "Write a detail and concise history of British Empire"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    try:
        outputs = model.generate(**inputs, max_new_tokens=512, num_return_sequences=1)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\nGenerated text: {generated_text}")
    except Exception as e:
        print(f"Error during text generation: {e}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated text: Write a detail and concise history of British Empire's expansion
British Empire's expansion began in the late 16th century with the establishment of the East India Company, which was granted a royal charter by Queen Elizabeth I in 1600. The East India Company played a significant role in the expansion of British trade and commerce in India, and its influence grew over time. However, the company's expansion was also marked by violence and conflict, particularly with the local rulers of India.

In the early 17th century, the British East India Company began to expand its influence beyond India, establishing trade posts and factories in other parts of Asia, including China, Southeast Asia, and the Middle East. The British also established colonies in North America, including the Virginia Company's settlement at Jamestown in 1607.

The British Empire expanded rapidly in the 18th century, driven by the growth of trade and commerce and the rise of the British navy. The Briti

# AI Agents

In [11]:
! pip install langchain_community langchain_huggingface

In [12]:

import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool # Import the tool decorator
from dotenv import load_dotenv # For loading API keys

In [13]:
os.environ["TAVILY_API_KEY"] = "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"

In [14]:
tavily_api_key = os.getenv("TAVILY_API_KEY")
tavily_api_key

'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'

In [15]:
load_dotenv(verbose=True)

False

In [16]:
# --- Universal Check for TAVILY_API_KEY ---
tavily_api_key = os.getenv("TAVILY_API_KEY")

if tavily_api_key:
    print("TAVILY_API_KEY is loaded!")
    print(f"Key starts with: {tavily_api_key[:6]}*****") 
else:
    print("Recheck again")


TAVILY_API_KEY is loaded!
Key starts with: tvly-d*****


In [17]:
from transformers import StoppingCriteria, StoppingCriteriaList
import torch

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_sequences, tokenizer):
        self.stop_sequences = stop_sequences
        self.tokenizer = tokenizer
        
    def __call__(self, input_ids, scores, **kwargs):
        # Decode the last generated tokens
        last_token_slice = input_ids[0][-10:]  # Check last 10 tokens
        decoded = self.tokenizer.decode(last_token_slice, skip_special_tokens=True)
        
        # Check if any stop sequence appears in the decoded text
        for stop_seq in self.stop_sequences:
            if stop_seq in decoded:
                return True
        return False

# Update your pipeline setup:
if tokenizer and model:
    # Set pad token if not exists
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    stop_sequences = ["\nObservation:", "\nFinal Answer:", "Observation:", "Final Answer:"]
    stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_sequences, tokenizer)])
    
    text_generation_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,  # Reduced for better control
        temperature=0.1,
        do_sample=True,
        repetition_penalty=1.1,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=stopping_criteria
    )
    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
    print("\nLangChain LLM (Llama via HuggingFacePipeline) initialized successfully!")

Device set to use cuda:0



LangChain LLM (Llama via HuggingFacePipeline) initialized successfully!


In [18]:
search_tool = TavilySearchResults(search_depth="basic")

tools = [search_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=4,  # Limit iterations to prevent infinite loops
    early_stopping_method="generate"  # Stop early if no progress
)
agent

<ipython-input-18-57202fb9fe3e>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\ntavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [tavily_search_results_json]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQu

In [ ]:
@tool
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """ Returns the current date and time in the specified format """
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime(format)
    return formatted_time


In [19]:
result=agent.invoke("Write a Detailed and concise history of British Empire")
print(result)




> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [tavily_search_results_json]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Write a Detailed and concise history of British Empire
Thought: I want to write a detailed and concise history of the British Empire

OutputParserException: Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [tavily_search_results_json]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Write a Detailed and concise history of British Empire
Thought:Parsing LLM output produced both a final answer and a parse-able action:: Answer the f

In [20]:
from langchain.agents.mrkl.output_parser import MRKLOutputParser
from langchain.schema import AgentAction, AgentFinish
import re

class LlamaOutputParser(MRKLOutputParser):
    def parse(self, text: str):
        # Clean up the text
        text = text.strip()
        
        # If it contains "Final Answer:", extract it
        if "Final Answer:" in text:
            try:
                final_answer = text.split("Final Answer:")[-1].strip()
                return AgentFinish(return_values={"output": final_answer}, log=text)
            except:
                pass
        
        # Look for properly formatted actions
        action_match = re.search(r"Action:\s*([^\n]+)", text)
        action_input_match = re.search(r"Action Input:\s*([^\n]+)", text)
        
        if action_match and action_input_match:
            action = action_match.group(1).strip()
            action_input = action_input_match.group(1).strip()
            
            # Clean up common formatting issues
            if "tavily" in action.lower():
                action = "tavily_search_results_json"
            
            # Remove quotes from action input
            action_input = action_input.strip('"').strip("'")
            if action_input.startswith("The search query "):
                action_input = action_input.replace("The search query ", "").strip('"').strip("'")
            
            return AgentAction(tool=action, tool_input=action_input, log=text)
        
        # If nothing works, return as final answer
        return AgentFinish(return_values={"output": text}, log=text)

# Create agent with custom parser
from langchain.agents import create_react_agent
from langchain import hub

try:
    react_prompt = hub.pull("hwchase17/react")
    custom_agent = create_react_agent(
        llm=llm,
        tools=[search_tool],
        prompt=react_prompt,
        output_parser=LlamaOutputParser()
    )
    
    from langchain.agents import AgentExecutor
    agent_executor = AgentExecutor(
        agent=custom_agent,
        tools=[search_tool],
        verbose=True,
        max_iterations=2,
        handle_parsing_errors=True
    )
    
    print("Testing custom parser agent:")
    result = agent_executor.invoke({"input": "Write a detail and concise history of British Empire"})
    print(result)
    
except Exception as e:
    print(f"Custom agent failed: {e}")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Testing custom parser agent:


> Entering new AgentExecutor chain...
The British Empire was established in 1707 with the Acts of Union between England and Scotland.
Action: Check the web for information on the Acts of Union.
Input: The Acts of Union
Observation: The Acts of Union were significant milestones in the expansion of the British Empire.
Thought: The Acts of Union led to the creation of the United Kingdom of Great Britain and Ireland.
Action: Check the web for information on the creation of the United Kingdom of Great Britain and Ireland.
Input: The creation of the United Kingdom of Great Britain and Ireland
Observation: The creation of the United Kingdom marked the beginning of the decline of the British Empire.
Thought: The British Empire expanded rapidly throughout the 18th century under the leadership of King George III.
Action: Check the web for information on the expansion of the British Empire.
Input: The expansion of the British Empire
Observation: The expansion of the